In [15]:
def __resample_convert_wav_custom(folder_in, sr=16000, ext='.wav'):
        
        # find total number of files to convert
        total_count = 0
        for (dirpath, _, filenames) in os.walk(folder_in):
            for filename in sorted(filenames):
                if filename.endswith(ext):
                    total_count += 1
        print(f"Total number of speech files to convert to 1-sec .wav: {total_count}")

        converted_count = 0
        # segment each audio file to 1-sec frames and save
        for (dirpath, _, filenames) in os.walk(folder_in):
            for filename in sorted(filenames):
                i = 0
                if filename.endswith(ext):
                    fname = os.path.join(dirpath, filename)
                    data, _ = librosa.load(fname, sr=sr)

                    # normalize data
                    mx = np.amax(abs(data))
                    data = data / mx

                    chunk_start = 0
                    frame_count = 0

                    precursor_len = 30 * 128
                    postcursor_len = 98 * 128
                    utternace_threshold = 30

                    while True:
                        if chunk_start + postcursor_len > len(data):
                            break

                        chunk = data[chunk_start: chunk_start + 128]
                        # scaled average over 128 samples
                        avg = 1000 * np.average(abs(chunk))
                        i += 128

                        if avg > utternace_threshold and chunk_start >= precursor_len:
                            print(f"\r Converting {converted_count + 1}/{total_count} "
                                f"to {frame_count + 1} segments", end=" ")
                            frame = data[chunk_start - precursor_len:chunk_start + postcursor_len]

                            # Using the directory path instead of a folder_out
                            outfile = os.path.join(dirpath, filename[:-4] + f'_chunk{frame_count}.wav')
                            sf.write(outfile, frame, sr)

                            chunk_start += postcursor_len
                            frame_count += 1
                        else:
                            chunk_start += 128
                    # Deleting the original large .wav file
                    os.remove(fname)
                    converted_count += 1
        print(f'\rFile conversion completed: {converted_count} files ')

In [16]:
import os
folder_in = r"C:\Users\JDiwanga\OneDrive - Analog Devices, Inc\Documents\PMO_ML\ML projects\Audio Classification\max78000fthr\smart echo\ai8x-training\data\raw"
sr=16000
ext='.wav'

# find total number of files to conv
__resample_convert_wav_custom(folder_in=folder_in)

Total number of speech files to convert to 1-sec .wav: 52
File conversion completed: 52 files 


In [9]:
import numpy as np
import librosa
import soundfile as sf
converted_count = 0
# segment each audio file to 1-sec frames and save
for (dirpath, _, filenames) in os.walk(folder_in):
    for filename in sorted(filenames):
        i = 0
        if filename.endswith(ext):
            fname = os.path.join(dirpath, filename)
            data, _ = librosa.load(fname, sr=sr)

            # normalize data
            mx = np.amax(abs(data))
            data = data / mx

            chunk_start = 0
            frame_count = 0

            precursor_len = 30 * 128
            postcursor_len = 98 * 128
            utternace_threshold = 30

            while True:
                if chunk_start + postcursor_len > len(data):
                    break

                chunk = data[chunk_start: chunk_start + 128]
                # scaled average over 128 samples
                avg = 1000 * np.average(abs(chunk))
                i += 128

                if avg > utternace_threshold and chunk_start >= precursor_len:
                    print(f"\r Converting {converted_count + 1}/{total_count} "
                        f"to {frame_count + 1} segments", end=" ")
                    frame = data[chunk_start - precursor_len:chunk_start + postcursor_len]

                    # Using the directory path instead of a folder_out
                    outfile = os.path.join(dirpath, filename[:-4] + f'_chunk{frame_count}.wav')
                    sf.write(outfile, frame, sr)

                    chunk_start += postcursor_len
                    frame_count += 1
                else:
                    chunk_start += 128
            os.remove(fname)
            converted_count += 1
print(f'\rFile conversion completed: {converted_count} files ')

File conversion completed: 52 files 
